# Creating an SQLite database

In this tutorial, we'll create a simple house price data database using SQLite.

## Prepare data for an SQL schema

First off, we use Pandas to take a look at the data we want to load into SQLite. We add a "market_id" column which is the unique identifier per row. Later we'll use this a primary key in a table.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/housing-data.csv")[:20]

## Add a market_id column which will become the primary key for the table below

df["market_id"] = [ f"prop_{i}" for i in range(len(df))]

df.head(2)

,price,bedrooms,bathrooms,estate_agent_code,transport_link_code,council_tax,freehold,garage,parking,market_id
0,304000,4,3,4,7,F,True,True,False,prop_0
1,218000,2,3,3,7,B,False,True,False,prop_1


## Prepare an empty SQL database

Next we create an empty database called "housing.db" which we store in the local file system. 

In [3]:
from sqlalchemy import create_engine
from pathlib import Path

In [4]:
database_relative_path = "data/housing.db"

database_path = Path.cwd().joinpath(database_relative_path)

engine = create_engine(f'sqlite:///{database_path}', echo=True)

## Create models

Then we create some models using sqlalchemy to map Python objects into the database schema. For a more in depth example, have a look at [this page](https://docs.sqlalchemy.org/en/20/orm/quickstart.html).

In [5]:
from typing import List
from typing import Optional
from sqlalchemy import ForeignKey
from sqlalchemy import String, Float, Integer
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class Base(DeclarativeBase):
    pass

class HousingMarket(Base):
    
    __tablename__ = "housing_market"
    
    market_id: Mapped[str] = mapped_column(primary_key=True)
    price: Mapped[float] = mapped_column(Float())
    bedrooms: Mapped[int] = mapped_column(Integer())
    bathrooms: Mapped[int] = mapped_column(Integer())
    estate_agent_code: Mapped[int] = mapped_column(Integer())
    transport_link_code: Mapped[int] = mapped_column(Integer())
    council_tax: Mapped[str] = mapped_column(String())
    freehold: Mapped[int] = mapped_column(Integer())
    garage: Mapped[int] = mapped_column(Integer()) 
    parking: Mapped[int] = mapped_column(Integer()) 
        
class TransportCodes(Base):
    
    __tablename__ = "transport_codes"
    
    transport_code: Mapped[int] = mapped_column(ForeignKey("housing_market.transport_link_code"), primary_key=True)
    transport_string: Mapped[str] = mapped_column(String())

        
class EstageAgentCodes(Base):
    
    __tablename__ = "estate_agent_codes"
    
    estate_agent_code: Mapped[int] = mapped_column(ForeignKey("housing_market.estate_agent_code"), primary_key=True)
    estate_agent_string: Mapped[str] = mapped_column(String())
        

In [6]:
Base.metadata.create_all(engine)

2023-02-26 17:02:44,001 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-26 17:02:44,003 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("housing_market")
2023-02-26 17:02:44,003 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-26 17:02:44,003 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("housing_market")
2023-02-26 17:02:44,003 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-26 17:02:44,003 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("transport_codes")
2023-02-26 17:02:44,003 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-26 17:02:44,003 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("estate_agent_codes")
2023-02-26 17:02:44,003 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-26 17:02:44,003 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("estate_agent_codes")
2023-02-26 17:02:44,003 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-02-26 17:02:44,003 INFO sqlalchemy.engine.Engine 
CREATE TABLE housing_market (
	market_id VARCHAR NOT NU

## Populate tables with data

The we populate our tables with data using the data from the DataFrame.

In [7]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    
    db_rows = []
    
    for index, row in df.iterrows():
        
        house_item = HousingMarket(
        
            market_id = row.market_id,
            price = row.price,
            bedrooms = row.bedrooms,
            bathrooms = row.bathrooms,
            estate_agent_code = row.estate_agent_code,
            transport_link_code = row.transport_link_code,
            council_tax = row.council_tax,
            freehold = row.freehold,
            garage = row.garage,
            parking = row.parking,
        )
        
        db_rows.append(house_item)
        
    session.add_all(db_rows)
    session.commit()

2023-02-26 17:02:44,038 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-26 17:02:44,038 INFO sqlalchemy.engine.Engine INSERT INTO housing_market (market_id, price, bedrooms, bathrooms, estate_agent_code, transport_link_code, council_tax, freehold, garage, parking) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2023-02-26 17:02:44,038 INFO sqlalchemy.engine.Engine [generated in 0.00069s] [('prop_0', 304000.0, 4, 3, 4, 7, 'F', True, True, False), ('prop_1', 218000.0, 2, 3, 3, 7, 'B', False, True, False), ('prop_2', 100000.0, 1, 2, 5, 8, 'C', True, False, True), ('prop_3', 758000.0, 4, 3, 4, 4, 'D', True, False, True), ('prop_4', 171000.0, 2, 2, 5, 2, 'B', True, False, False), ('prop_5', 764000.0, 5, 5, 5, 7, 'F', True, False, True), ('prop_6', 369000.0, 4, 3, 3, 7, 'D', True, False, True), ('prop_7', 158000.0, 2, 2, 2, 1, 'A', False, False, True)  ... displaying 10 of 20 total bound parameter sets ...  ('prop_18', 382000.0, 3, 5, 4, 6, 'F', True, True, False), ('prop_19', 128000.0, 2, 2, 5

In [8]:
estate_agent_mapping = {
    
    1: "Clapham",
    2: "Deptford",
    3: "Brixton",
    4: "Richmond",
    5: "Isle of Dogs"
}

In [9]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    
    db_rows = []
    
    for code in estate_agent_mapping:
        
        estate_agent_code = EstageAgentCodes(
        
            estate_agent_code = code,
            estate_agent_string = estate_agent_mapping.get(code),
        )
        
        db_rows.append(estate_agent_code)
        
    session.add_all(db_rows)
    session.commit()

2023-02-26 17:02:44,075 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-26 17:02:44,075 INFO sqlalchemy.engine.Engine INSERT INTO estate_agent_codes (estate_agent_code, estate_agent_string) VALUES (?, ?)
2023-02-26 17:02:44,075 INFO sqlalchemy.engine.Engine [generated in 0.00043s] [(1, 'Clapham'), (2, 'Deptford'), (3, 'Brixton'), (4, 'Richmond'), (5, 'Isle of Dogs')]
2023-02-26 17:02:44,075 INFO sqlalchemy.engine.Engine COMMIT


## Query data from database

Finally, to verify that the data has been loaded correctly, we run a test query.

In [10]:
sql_query_string = """
SELECT 
eac.estate_agent_string,
hm.market_id,
hm.price, 
hm.bedrooms 
FROM housing_market hm
JOIN estate_agent_codes eac on (hm.estate_agent_code = eac.estate_agent_code)
where eac.estate_agent_string in ("Richmond", "Clapham")

"""

In [11]:
from sqlalchemy import sql
from  sqlalchemy.exc import OperationalError
from tabulate import tabulate

str_sql = sql.text(sql_query_string)

with Session(engine) as session:
    
    query = session.execute(str_sql)
    
    query_results = query.fetchall()
        
    print(tabulate(query_results, headers=query.keys(), tablefmt='sqlite'))

2023-02-26 17:02:44,119 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-26 17:02:44,120 INFO sqlalchemy.engine.Engine 
SELECT 
eac.estate_agent_string,
hm.market_id,
hm.price, 
hm.bedrooms 
FROM housing_market hm
JOIN estate_agent_codes eac on (hm.estate_agent_code = eac.estate_agent_code)
where eac.estate_agent_string in ("Richmond", "Clapham")


2023-02-26 17:02:44,122 INFO sqlalchemy.engine.Engine [generated in 0.00106s] ()
estate_agent_string    market_id      price    bedrooms
---------------------  -----------  -------  ----------
Richmond               prop_0        304000           4
Richmond               prop_3        758000           4
Richmond               prop_8        228000           4
Richmond               prop_9        209000           2
Richmond               prop_16       313000           3
Richmond               prop_18       382000           3
2023-02-26 17:02:44,122 INFO sqlalchemy.engine.Engine ROLLBACK


## Drop tables

Then we drop the table, to keep the example idempotent for the next run.

In [12]:
def drop_tables(table_name):

    str_sql = sql.text(f"drop table {table_name}")

    with Session(engine) as session:

        try:

            query = session.execute(str_sql)

        except OperationalError as e:

            print(e)
            

drop_tables("estate_agent_codes")
drop_tables("housing_market")

2023-02-26 17:02:44,140 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-26 17:02:44,140 INFO sqlalchemy.engine.Engine drop table estate_agent_codes
2023-02-26 17:02:44,140 INFO sqlalchemy.engine.Engine [generated in 0.00083s] ()
2023-02-26 17:02:44,153 INFO sqlalchemy.engine.Engine ROLLBACK
2023-02-26 17:02:44,154 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-02-26 17:02:44,156 INFO sqlalchemy.engine.Engine drop table housing_market
2023-02-26 17:02:44,156 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ()
2023-02-26 17:02:44,156 INFO sqlalchemy.engine.Engine ROLLBACK
